In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("Spark SQL Advanced Functions").getOrCreate()


In [2]:
data = [
    ("Alice", "Math", 90),
    ("Alice", "Science", 85),
    ("Bob", "Math", 70),
    ("Bob", "Science", 80),
    ("Charlie", "Math", 95),
    ("Charlie", "Science", 91)
]

columns = ["name", "subject", "score"]

df = spark.createDataFrame(data, columns)
df.show()


+-------+-------+-----+
|   name|subject|score|
+-------+-------+-----+
|  Alice|   Math|   90|
|  Alice|Science|   85|
|    Bob|   Math|   70|
|    Bob|Science|   80|
|Charlie|   Math|   95|
|Charlie|Science|   91|
+-------+-------+-----+



In [3]:
# Sort by name ascending and score descending
df_sorted = df.orderBy("name", col("score").desc())
df_sorted.show()


+-------+-------+-----+
|   name|subject|score|
+-------+-------+-----+
|  Alice|   Math|   90|
|  Alice|Science|   85|
|    Bob|Science|   80|
|    Bob|   Math|   70|
|Charlie|   Math|   95|
|Charlie|Science|   91|
+-------+-------+-----+



In [4]:
df_agg = df.groupBy("name").agg(
    count("*").alias("total_subjects"),
    avg("score").alias("average_score"),
    sum("score").alias("total_score"),
    max("score").alias("max_score"),
    min("score").alias("min_score")
)
df_agg.show()


+-------+--------------+-------------+-----------+---------+---------+
|   name|total_subjects|average_score|total_score|max_score|min_score|
+-------+--------------+-------------+-----------+---------+---------+
|    Bob|             2|         75.0|        150|       80|       70|
|  Alice|             2|         87.5|        175|       90|       85|
|Charlie|             2|         93.0|        186|       95|       91|
+-------+--------------+-------------+-----------+---------+---------+



In [5]:
window_spec = Window.partitionBy("name").orderBy("score")

df_window = df.withColumn("rank", rank().over(window_spec)) \
              .withColumn("dense_rank", dense_rank().over(window_spec)) \
              .withColumn("row_number", row_number().over(window_spec)) \
              .withColumn("avg_score_over", avg("score").over(window_spec))

df_window.show()


+-------+-------+-----+----+----------+----------+--------------+
|   name|subject|score|rank|dense_rank|row_number|avg_score_over|
+-------+-------+-----+----+----------+----------+--------------+
|  Alice|Science|   85|   1|         1|         1|          85.0|
|  Alice|   Math|   90|   2|         2|         2|          87.5|
|    Bob|   Math|   70|   1|         1|         1|          70.0|
|    Bob|Science|   80|   2|         2|         2|          75.0|
|Charlie|Science|   91|   1|         1|         1|          91.0|
|Charlie|   Math|   95|   2|         2|         2|          93.0|
+-------+-------+-----+----+----------+----------+--------------+



In [6]:
json_data = [
    ('{"name":"Alice","age":21,"city":"Delhi"}',),
    ('{"name":"Bob","age":22,"city":"Mumbai"}',),
    ('{"name":"Charlie","age":23,"city":"Chennai"}',)
]

json_df = spark.createDataFrame(json_data, ["json_string"])

# Parse JSON string into columns
df_json = json_df.select(
    from_json("json_string", "name STRING, age INT, city STRING").alias("parsed")
).select("parsed.*")

df_json.show()


+-------+---+-------+
|   name|age|   city|
+-------+---+-------+
|  Alice| 21|  Delhi|
|    Bob| 22| Mumbai|
|Charlie| 23|Chennai|
+-------+---+-------+

